# Algorithme des contraintes actives
Par Julien Corriveau-Trudel

In [8]:
using LinearAlgebra;
import Pkg; Pkg.add("ForwardDiff");
using ForwardDiff;
IJulia.clear_output();

# Algorithme

## Algorithme des contraintes actives
L'objectif de cet algorithme est de résoudre un programme non-linéaire avec contraintes d'inégalités linéaires.

In [61]:
# ActSet : Algorithme des contraintes actives
#
# Entrée: f (Function):
#         x0: Point initial, vecteur colonne
#         A (Array{Float64,2}): Matrice de contraintes, tel que Ax =< b.
#         b : Vecteur de contraintes, tel que Ax =< b.
#         τ (Float64 ∈[0,0.5]): paramètre de pas admissible pour le critère d'Armijo
#         arrêt: nombre d'itération max, afin que l'algorithme prenne fin
#         ϵ: critère d'arrêt, valeur min de Z∇f(x) avant de le considérer nul
# Sortie: Point considéré comme minimum local

function ActSet(f::Function, x0, A::Array{Float64,2}, b, arrêt = Inf, τ = 0.1,  ϵ::Float64 = sqrt(eps()))
    if size(x0)[1] == 1
        if size(x0)[2] != 1
            return ArgumentError("Argument x0 vecteur ligne. Doit être vecteur colonne.")
        end
    end
    if size(b)[1] == 1
        if size(b)[2] != 1
            return ArgumentError("Argument b vecteur ligne. Doit être vecteur colonne.")
        end
    end
    
    for i in 1:size(A)[1]
        if (A[[i],:]*x0)[1] - b[i] > ϵ
            return ArgumentError("x0 n'est pas un point réalisable.")
        end
    end
    
    
    # Dimensions des choses
    #n = size(x0)[1]
    #m = size(A)[1]
    
    # Trouver l'ensemble de contraintes actives
    W = FindActiveSet(x0, A, b, ϵ)
    # Trouver le noyau de A_W
    Z = nullspace(A[W,:])
    
    # Gradient de la fonction
    ∇f = x -> ForwardDiff.gradient(f, x);
    xk = x0
    
    # Itération
    it_restr = 0
    it_relax = 0
    # Grande boucle
    while it_restr < arrêt
        # RESTRICTION
        while norm(∇f(xk)'*Z) > ϵ
            #print("Restriction. \n")
            it_restr = it_restr + 1

            # Calcul de sk, direction de descente de minϕ(α) = f(xk + Z*α)
            sk = -(Z'*∇f(xk))
            dk = Z*sk
            θk = Pas_Admissible(xk, dk, f, τ, ∇f(xk)')
            θmax = Findθmax(xk, dk, A, b, ϵ)

            if θmax <= θk
                xk = xk + θmax*dk
                W = FindActiveSet(xk, A, b, ϵ)
                Z = nullspace(A[W,:])
            else
                xk = xk + θk*dk
            end
            
            if it_restr >= arrêt
                print("Arrêt prématurée à ",it_restr, " it de restriction et " ,it_relax," it de relaxation. \n")
                return xk
            end
        end
        # RELAXATION
        it_relax = it_relax + 1
        if size(W)[1] >= 1
            
            dim_A = size(A[W,:])
            
            if (dim_A[1] == dim_A[2])&(rank(A[W,:]) !== dim_A[2])
                λ=[A[W,:];zeros(dim_A[2])']\[-∇f(xk);0]
            else
                λ = (A[W,:]')\-∇f(xk)
            end
            #print("λ: ",λ, "\n")
        elseif size(W)[1] == 0
            print("Arrêt optimal à ",it_restr, " it de restriction et " ,it_relax," it de relaxation. \n")
            return xk;
        end
        
        λmin = minimum(λ)
        
        if λmin >= -ϵ
            print("Arrêt optimal à ",it_restr, " it de restriction et " ,it_relax," it de relaxation. \n")
            return xk;
        end
        
        
        # ENLEVER LA PLUS PETITE
        #ind = findfirst(isequal(λmin) , λ)
        #W = deleteat!(W,ind)
        
        # ENLEVER UNE NÉGATIVE ALÉATOIREMENT
        #NbNég = 0
        #for j in 1:size(W)[1]
        #    if λ[j] < 0.
        #        NbNég = NbNég + 1
        #    end
        #end
        #ÀEnlever = Int64(floor(rand()*NbNég + 1))
        #for j in 1:size(W)[1]
        #    if λ[j] < 0.
        #        ÀEnlever = ÀEnlever - 1
        #    end
        #    if ÀEnlever == 0
        #        deleteat!(W,j)
        #        break
        #    end
        #end
        
        # ENLEVER LA PLUS PETITE, AVEC BRIS D'ÉGALITÉ ALÉATOIRE
        PP = 0
        #print("W: ", W, "\n")
        for j in 1:size(W)[1]
            if abs(λ[j] - λmin) < ϵ
                PP = PP + 1
            end
        end
        
        if(PP != 1)
            ÀEnlever = Int64(floor(rand()*PP + 1))
            for j in 1:size(W)[1]
                if abs(λ[j] - λmin) < ϵ
                    ÀEnlever = ÀEnlever - 1
                end
                if ÀEnlever == 0
                    #print("On retire: ", j, "\n")
                    deleteat!(W,j)
                    break
                end
            end
        else
            ind = findfirst(isequal(λmin) , λ)
            W = deleteat!(W,ind)
        end
        
        #W = deleteat!(W,ind)
        Z = nullspace(A[W,:])
    end
end;

In [3]:
# FindActiveSet: renvoie les indices des contraintes qui sont respectées avec égalités, ordonnées
function FindActiveSet(x, A, b, ϵ)
    actSet = []
    for i in 1:size(A)[1]
        if abs((A[i,:]'*x)[1] - b[i]) < ϵ
            actSet = push!(actSet, i)
        end
    end
    return sort(actSet)
end;

# Findθmax: trouve θmax
function Findθmax(x, dk, A, b, ϵ)
    θ = []
    as = FindActiveSet(x,A,b,ϵ)
    for i in 1:size(A)[1]
        if i in as
        else
            tmp1 = (b[i]-A[i,:]'*x)[1]
            tmp2 = (A[i,:]'*dk)[1]
            if tmp2 > 0 # θ est borné supérieurement seulement si ai*dk > 0 
                θ = push!(θ, tmp1/tmp2)
            end
        end
    end
    if size(θ)[1] > 0
        return minimum(θ)
    else
        return Inf
    end
end;

# On utilisera la mise en oeuvre d'un pas admissible tel que vu à la page 150 des NdC de J.-P. Dussault.
function Pas_Admissible(x, d, f::Function, τ, ∇f)
    θ = 1.0
    while f(x+θ*d)- f(x) > θ*τ*(∇f*d)[1]
        θ = θ/2
    end
    return θ
end;

# Tests

## Validation avec le problème #2
On valide premièrement l'algorithme avec le programme de l'exercice 2, sachant que l'exercice 1 admettra une infinité de multiplicateurs pour la solution optimale, ce qui ajoute une complexité de résolution pour l'algorithme. Commençons par énoncer le programme:

$$ \max{f(x) = (x_1 - 1)^2(6 - x_2)}$$
\begin{align}
\text{s.à.}& -x_1 &\leq -2\\
& x_1 - m x_2 &\leq 0\\
& -x_2 & \leq 0 \\
& x_2 & \leq 5\\
\end{align}

avec $m > 0$. Le problème de maximisation est équivalent à celui de minimisation:
$$ \min{-f(x) = (x_1 - 1)^2(x_2- 6)}$$
\begin{align}
\text{s.à.}& -x_1 &\leq -2\\
& x_1 - m x_2 &\leq 0\\
& -x_2 & \leq 0 \\
& x_2 & \leq 5\\
\end{align}

### Choix de point de départ $x_0$
Le point de départ doit être dans l'espace convexe de valeurs réalisables. On testera les points de départs suivant, pour chaque valeur de $m$:

``x0s = ([2.,5.], [2. + δ1, 5. - δ2], [2., 5. - δ2])``,

avec $\delta 1$ et $\delta2$ tels que le point reste dans le domaine réalisable, sans être sur une contrainte.
Ainsi, nous avons un point qui commence sur une, deux ou aucune contrainte des contraites.


### Choix de m 
On sait que pour $m < \frac{2}{5}$, il n'existe aucune solution au programme. Ainsi, on passera l'algorithme plusieurs fois sur une grille de $m$ qui réflète les différents cas trouvés lors de l 'exercice #2. Voici les valeurs de m choisies:



In [52]:
ms = [49. /120., 3. /4., 1., 5. /2., 15., 113.]

6-element Array{Float64,1}:
   0.4083333333333333
   0.75              
   1.0               
   2.5               
  15.0               
 113.0               

Voici les trois cas de figure à laquel on s'attend:
### Cas 1 $m \in [\frac{2}{5}, \frac{5}{12}]$
Le point optimal attendu est: $x^* = (2, \frac{2}{m})^t$.

### Cas 2 $m \in [\frac{5}{12}, 1]$
La théorie obtenu lors de la résolution de l'exercice 2 nous suggère qu'il n'y aurait pas de solution. Toutefois, nous sommes conscient que ce résultat est erroné. 

### Cas 3 $m \in [1, \infty)$
Le point optimal attendu est: $x^* = (4m + \frac{1}{3}, 4 + \frac{1}{3m})^t$.


On pourra comparer les points obtenus par l'algorithme avec ce que la théorie nous dit.


In [42]:
f2(x::Vector) = (x[1]-1)^2*(x[2] - 6)

b2 = [-2.; 0.; 0.; 5.]

#res = []

for i in 1:size(ms)[1]
    m = ms[i]
    δ1 = 0.03
    δ2 = 0.02
    A2 = [-1. 0.; 1. -m; 0. -1.; 0. 1.]
    x0s = ([2.,5.], [2. + δ1, 5. - δ2], [2., 5. - δ2])
    for xinit in x0s    
        print("Soln pour m = ", m ,", x0 = ",xinit,": ", ActSet(f2, xinit, A2, b2, 10000), "\n")
    end
end

Arrêt optimal à 2 it de restriction et 3 it de relaxation. 
Soln pour m = 0.4083333333333333, x0 = [2.0, 5.0]: [2.0, 4.89796]
Arrêt optimal à 2 it de restriction et 1 it de relaxation. 
Soln pour m = 0.4083333333333333, x0 = [2.03, 4.98]: [2.0, 4.89796]
Arrêt optimal à 1 it de restriction et 1 it de relaxation. 
Soln pour m = 0.4083333333333333, x0 = [2.0, 4.98]: [2.0, 4.89796]
Arrêt optimal à 36 it de restriction et 3 it de relaxation. 
Soln pour m = 0.75, x0 = [2.0, 5.0]: [3.33333, 4.44444]
Arrêt optimal à 42 it de restriction et 1 it de relaxation. 
Soln pour m = 0.75, x0 = [2.03, 4.98]: [3.33333, 4.44444]
Arrêt optimal à 39 it de restriction et 3 it de relaxation. 
Soln pour m = 0.75, x0 = [2.0, 4.98]: [3.33333, 4.44444]
Arrêt prématurée à 10000 it de restriction et 2 it de relaxation. 
Soln pour m = 1.0, x0 = [2.0, 5.0]: [4.33333, 4.33333]
Arrêt prématurée à 10000 it de restriction et 0 it de relaxation. 
Soln pour m = 1.0, x0 = [2.03, 4.98]: [4.33333, 4.33333]
Arrêt prématurée à 

La théorie nous indiquait que les points qu'on devait trouver sont:

In [53]:
print([2, 2. /(49. /120.)], "\n")
print("Aucune réponse pour m = 0.75. \n")
for m in deleteat!(ms, [1 2])
    print([4*m + 1/3., 4 + 1/(3. * m)],"\n")
end

[2.0, 4.89796]
Aucune réponse pour m = 0.75. 
[4.33333, 4.33333]
[10.3333, 4.13333]
[60.3333, 4.02222]
[452.333, 4.00295]


## Analyse des résultats
L'algorithme a trouvé les points attendus pour presque tous les points. En effet, pour $m \in \{\frac{49}{120}, , 1, \frac{5}{2}, 113\}$, l'algorithme a trouvé les points prévus, peu importe le point initial. Par contre, pour les deux autres valeurs de $m$, il faut voir ce qu'il se passe.

Nous sommes conscient que dans la démarche du devoir, le fait que l'intervalle $(5/12, 1)$ ne soit pas couvert était incongrue, ce que l'algorithme nous démontre plus haut, pour $ m = \frac{3}{4}$. Toutefois, en utilisant la même règle que pour le cas où $m \in [1, \infty)$, le point obtenu concorde! 


In [56]:
[4*(3/4) + 1/3., 4 + 1/(3. * (3/4))]

2-element Array{Float64,1}:
 3.3333333333333335
 4.444444444444445 

Pour $m = 15$, un phénomène intéressant survient: le résultat ne concorde pas avec la règle obtenu lors de la théorie.
Nous obtenons plutôt deux résultats qui diffèrent selon le point de départ: $(72.2896, 4.81931)^t$ et $(2.0, 0.133333)^t$. Le premier a été obtenu avec les points initiaux $x_0 = (2, 5)^t$ et $x_0 = (2.0, 4.98)^t$, alors que l'autre est obtenu en commençant à $x_0 = (2.03, 4.98)^t$. Ni l'un ni l'autre ne colle à la règle: $x^* = (4m + \frac{1}{3}, 4 + \frac{1}{3m})^t$.

C'est fort probablement un cas particulier qui nous a échappé lors de la théorie, voyant la performance de l'algorithme pour les autres valeurs de $m$.

## Validation avec le problème #2
On valide ensuite l'algorithme avec le programme de l'exercice 1. Nous savons par la théorie que ce problème admettra une infinité de multiplicateurs pour une solution optimale en $x^* = (0, \frac{1}{2}, \frac{1}{2})^t$, ce qui ajoute une complexité de résolution pour l'algorithme. Commençons par énoncer le programme:

$$ \min{f(x) = \frac{1}{2}(x_1^2 + 6x_1 + 2x_2^2 + 2 x_3^2)}$$
\begin{align}
\text{s.à. }\quad \quad& x_1 + x_2 + x_3 &= 1\\
& x_1, x_2, x_3 &\leq 0\\
\end{align}


### Choix de point de départ $x_0$
Le point de départ doit être dans l'espace convexe de valeurs réalisables. On testera des points sur une, deux ou trois contrainte, en considérant la contrainte d'égalité comme étant une seule contrainte, en plus de commencer avec le point optimal local $x^* = (0, \frac{1}{2}, \frac{1}{2})^t$:


In [57]:
x0s2 = ([1. /3.; 1. /3.; 1. /3.], [0.5, 0, 0.5], [1.,0., 0.],[0, 0.5, 0.5])

([0.333333, 0.333333, 0.333333], [0.5, 0.0, 0.5], [1.0, 0.0, 0.0], [0.0, 0.5, 0.5])

In [58]:
#Premiers Tests de la fonction
f(x::Vector) = (x[1]^2+6*x[1]+2*x[2]^2 + 2*x[3]^2)/2
A = [1. 1. 1.;-1. -1. -1.; -1. 0. 0. ; 0. -1. 0.; 0. 0. -1.]
b = [1.,-1.,0.,0.,0.]
for xinit in x0s2
    print("Soln pour x0 = ",xinit,": ", ActSet(f, xinit, A, b, 10000), "\n")
end

Arrêt optimal à 2 it de restriction et 3 it de relaxation. 
Soln pour x0 = [0.333333, 0.333333, 0.333333]: [0.0, 0.0, 0.0]
Arrêt optimal à 3 it de restriction et 4 it de relaxation. 
Soln pour x0 = [0.5, 0.0, 0.5]: [0.0, 0.0, 0.0]
Arrêt optimal à 3 it de restriction et 5 it de relaxation. 
Soln pour x0 = [1.0, 0.0, 0.0]: [0.0, 0.0, 0.0]
Arrêt optimal à 4 it de restriction et 4 it de relaxation. 
Soln pour x0 = [0.0, 0.5, 0.5]: [0.0, 0.0, 0.0]


## Analyse des résultats
Pour tous les points de départ, l'algorithme renvoit le point optimal $x^{**} = (0,0,0)$, même lorsqu'on commence par l'autre point optimal $x^* = (0, \frac{1}{2}, \frac{1}{2})^t$. Ceci est dû au fait que pour résoudre le système d'équation qui donne les multiplicateurs de Lagrange, on procède à une résolution qui donne une solution unique, alors que le système admet une infinité de solution pour les multiplicateurs de Lagrange au point $x^* = (0, \frac{1}{2}, \frac{1}{2})^t$. Ainsi, l'algorithme doit donne une solution qui possède des multiplicateurs négatifs, même s'il existe des solutions où les multiplicateurs sont tous positifs, ce qui fait penser à l'algorithme que ce point n'est pas une solution optimale.

Sinon, l'algorithme fonctionne bien.